In [1]:
import torch
import torch.optim as optim

import numpy as np
import pandas as pd
import time

In [2]:
torch.manual_seed(42) #Seed 고정

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')
submit = pd.read_csv('submit_sample.csv')

In [4]:
train_x = train.drop(['year', 'avgPrice'], axis=1)
train_y = pd.DataFrame(train.avgPrice)

test_x = test.drop(['year'], axis =1)

In [5]:
train_x

,avgTemp,minTemp,maxTemp,rainFall
0,-4.9,-11.0,0.9,0.0
1,-3.1,-5.5,5.5,0.8
2,-2.9,-6.9,1.4,0.0
3,-1.8,-5.1,2.2,5.9
4,-5.2,-8.7,-1.8,0.7
...,...,...,...,...
2552,1.2,-3.2,4.5,2.1
2553,-1.4,-6.5,4.8,0.0
2554,-1.3,-4.5,2.1,0.2
2555,-1.4,-7.0,4.6,0.0


In [6]:
train_x_new = train.drop(['year', 'avgPrice'], axis=1)
test_x_new = test.drop(['year'], axis =1)

In [7]:
train_x_new = train_x_new.shift().fillna(0)
test_x_new = test_x_new.shift().fillna(0)

In [8]:
train_x_new = train_x - train_x_new
test_x_new = test_x - test_x_new

In [9]:
train_x_new.loc[0, :]= 0
test_x_new.loc[0, :]= 0

In [10]:
train_x_new

,avgTemp,minTemp,maxTemp,rainFall
0,0.0,0.0,0.0,0.0
1,1.8,5.5,4.6,0.8
2,0.2,-1.4,-4.1,-0.8
3,1.1,1.8,0.8,5.9
4,-3.4,-3.6,-4.0,-5.2
...,...,...,...,...
2552,-3.4,-6.1,-1.7,-8.2
2553,-2.6,-3.3,0.3,-2.1
2554,0.1,2.0,-2.7,0.2
2555,-0.1,-2.5,2.5,-0.2


In [11]:
train_x = pd.concat([train_x, train_x_new], axis =1)
test_x = pd.concat([test_x, test_x_new], axis =1)

In [12]:
train_x

,avgTemp,minTemp,maxTemp,rainFall,avgTemp,minTemp,maxTemp,rainFall
0,-4.9,-11.0,0.9,0.0,0.0,0.0,0.0,0.0
1,-3.1,-5.5,5.5,0.8,1.8,5.5,4.6,0.8
2,-2.9,-6.9,1.4,0.0,0.2,-1.4,-4.1,-0.8
3,-1.8,-5.1,2.2,5.9,1.1,1.8,0.8,5.9
4,-5.2,-8.7,-1.8,0.7,-3.4,-3.6,-4.0,-5.2
...,...,...,...,...,...,...,...,...
2552,1.2,-3.2,4.5,2.1,-3.4,-6.1,-1.7,-8.2
2553,-1.4,-6.5,4.8,0.0,-2.6,-3.3,0.3,-2.1
2554,-1.3,-4.5,2.1,0.2,0.1,2.0,-2.7,0.2
2555,-1.4,-7.0,4.6,0.0,-0.1,-2.5,2.5,-0.2


In [13]:
train_x = torch.FloatTensor(train_x.values) #2557,8
train_y = torch.FloatTensor(train_y.values) #2557

test_x = torch.FloatTensor(test_x.values)

In [14]:
W = torch.zeros((8,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [15]:
optimizer = optim.SGD([W,b], lr=0.1)
total_epoch= 1000000
for epoch in range(1,total_epoch+1):
    hypothesis = train_x.matmul(W)+b
    cost = torch.mean((hypothesis - train_y)**2)
    cost = torch.sqrt(cost)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, total_epoch, cost.item()
        ))

Epoch 100000/1000000 Cost: 1381.244385
Epoch 200000/1000000 Cost: 1357.845947
Epoch 300000/1000000 Cost: 1344.729248
Epoch 400000/1000000 Cost: 1337.413696
Epoch 500000/1000000 Cost: 1333.344238
Epoch 600000/1000000 Cost: 1331.072632
Epoch 700000/1000000 Cost: 1329.808350
Epoch 800000/1000000 Cost: 1329.092651
Epoch 900000/1000000 Cost: 1328.684448
Epoch 1000000/1000000 Cost: 1328.439087


In [16]:
W

tensor([[-158.4249],
        [  46.0054],
        [ 131.7181],
        [   8.7482],
        [  62.6685],
        [ -30.7537],
        [ -59.5999],
        [  -4.8722]], requires_grad=True)

In [17]:
predict = test_x.matmul(W)+b

In [18]:
submit.Expected = predict.detach().numpy()

In [19]:
submit.to_csv('2021_09_05_02.csv', index=False)